In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql import types
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.window import Window
import os
import sys

In [2]:
year = sys.argv[1]
month = sys.argv[2]

In [3]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('data_transformation') \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 15:45:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data_path = '../raw_data'

In [5]:
new_schema = types.StructType([
    types.StructField('VendorID', types.IntegerType(), True), 
    types.StructField('tpep_pickup_datetime', types.TimestampNTZType(), True), 
    types.StructField('tpep_dropoff_datetime', types.TimestampNTZType(), True), 
    types.StructField('passenger_count', types.IntegerType(), True), 
    types.StructField('trip_distance', types.FloatType(), True), 
    types.StructField('RatecodeID', types.IntegerType(), True), 
    types.StructField('store_and_fwd_flag', types.StringType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('payment_type', types.IntegerType(), True), 
    types.StructField('fare_amount', types.FloatType(), True), 
    types.StructField('extra', types.FloatType(), True), 
    types.StructField('mta_tax', types.FloatType(), True), 
    types.StructField('tip_amount', types.FloatType(), True), 
    types.StructField('tolls_amount', types.FloatType(), True), 
    types.StructField('improvement_surcharge', types.FloatType(), True), 
    types.StructField('total_amount', types.FloatType(), True), 
    types.StructField('congestion_surcharge', types.FloatType(), True), 
    types.StructField('airport_fee', types.IntegerType(), True)])

In [41]:
df = spark.read.option('headers', True).parquet(f'{data_path}/2019/*/*.parquet')

In [42]:
old_schema = df.schema

In [43]:
for old_field, new_field in zip(old_schema.fields, new_schema.fields):
    df = df.withColumn(new_field.name, col(old_field.name).cast(new_field.dataType))

In [44]:
df = df.withColumnRenamed('VendorID', 'vendor_id') \
    .withColumnRenamed('RatecodeID', 'ratecode_id') \
    .withColumnRenamed('payment_type', 'payment_type_id') \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime') \
    .withColumnRenamed('PULocationID', 'pickup_location_id') \
    .withColumnRenamed('DOLocationID', 'dropoff_location_id')

In [45]:
df = df.filter((col('fare_amount') > 0) \
               & (col('trip_distance') > 0) \
               & (col('extra') > 0))

df = df.filter((col('ratecode_id') <= 6))

df = df.withColumn('congestion_surcharge', F.when(col('congestion_surcharge').isNull(), 0).otherwise(col('congestion_surcharge')))
df = df.withColumn('airport_fee', F.when(col('airport_fee').isNull(), 0).otherwise(col('airport_fee')))

In [46]:
window_spec = Window.orderBy('passenger_count')

df_rn = df.select(['passenger_count']).withColumn('rn', F.row_number().over(window_spec))
total_rows = df.count()
                                                         
if total_rows % 2 == 0:
    lower_mid = total_rows // 2
    upper_mid = lower_mid + 1
else:
    lower_mid = total_rows // 2 + 1
    upper_mid = lower_mid

median_df = df_rn.filter((col('rn') == lower_mid) | (col('rn') == upper_mid))

median_value = median_df.agg(F.avg(col('passenger_count'))).collect()[0][0]

df = df.withColumn('passenger_count', F.when(col('passenger_count') == 0, median_value).otherwise(col('passenger_count')))

24/09/09 18:59:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/09 18:59:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/09 18:59:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/09 18:59:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py",

KeyboardInterrupt: 

In [ ]:
def index_id(date_column):
    year = F.year(date_column)
    month = F.lpad(F.month(date_column).cast("string"), 2, "0")
    day = F.lpad(F.dayofmonth(date_column).cast("string"), 2, "0")
    hour = F.lpad(F.hour(date_column).cast("string"), 2, "0")
    minute = F.lpad(F.minute(date_column).cast("string"), 2, "0")
    second = F.lpad(F.second(date_column).cast("string"), 2, "0")
    index = F.concat(year, month, day, hour, minute, second)
    return index

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(Unknown Source)
	at java.base/sun.nio.ch.NioSocketImpl.accept(Unknown Source)
	at java.base/java.net.ServerSocket.implAccept(Unknown Source)
	at java.base/java.net.ServerSocket.platformImplAccept(Unknown Source)
	at java.base/java.net.ServerSocket.implAccept(Unknown Source)
	at java.base/java.net.ServerSocket.implAccept(Unknown Source)
	at java.base/java.net.ServerSocket.accept(Unknown Source)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


In [39]:
pickup_datetime_dim = df.select(['pickup_datetime']) \
    .distinct() \
    .withColumn('pickup_datetime_id', index_id(col('pickup_datetime'))) \
    .withColumn('pickup_hour', F.hour(col('pickup_datetime'))) \
    .withColumn('pickup_day', F.dayofmonth(col('pickup_datetime'))) \
    .withColumn('pickup_month', F.month(col('pickup_datetime'))) \
    .withColumn('pickup_year', F.year(col('pickup_datetime'))) \
    .withColumn('pickup_weekday', F.date_format(col('pickup_datetime'), 'EEEE'))

pickup_datetime_dim = pickup_datetime_dim.select(
    'pickup_datetime_id',
    'pickup_datetime',
    'pickup_hour',
    'pickup_day',
    'pickup_month',
    'pickup_year',
    'pickup_weekday'
)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `pickup_datetime` cannot be resolved. Did you mean one of the following? [`pickup_datetime_id`, `pickup_location_id`, `dropoff_datetime_id`, `airport_fee`, `tip_amount`].;
'Project ['pickup_datetime]
+- Project [vendor_id#459, pickup_datetime_id#739, dropoff_datetime_id#942, pickup_location_id#559, dropoff_location_id#579, ratecode_id#479, passenger_count#677, trip_distance#119, payment_type_id#499, store_and_fwd_flag#159, fare_amount#239, extra#259, mta_tax#279, tip_amount#299, tolls_amount#319, improvement_surcharge#339, congestion_surcharge#599, airport_fee#619, total_amount#359]
   +- Project [vendor_id#459, pickup_datetime#519, dropoff_datetime#539, passenger_count#677, trip_distance#119, ratecode_id#479, store_and_fwd_flag#159, pickup_location_id#559, dropoff_location_id#579, payment_type_id#499, fare_amount#239, extra#259, mta_tax#279, tip_amount#299, tolls_amount#319, improvement_sur

In [40]:
pickup_datetime_id_dict = {row['pickup_datetime']: row['pickup_datetime_id'] for row in pickup_datetime_dim.collect()} 

pickup_datetime_id_broadcast = sc.broadcast(pickup_datetime_id_dict)

def get_pickup_datetime_id(date_time):
    return pickup_datetime_id_broadcast.value.get(date_time, None)

get_pickup_datetime_id_udf = F.udf(get_pickup_datetime_id, types.IntegerType())

df = df.withColumn('pickup_datetime_id', get_pickup_datetime_id_udf('pickup_datetime'))

pickup_datetime_id_broadcast.unpersist()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `pickup_datetime` cannot be resolved. Did you mean one of the following? [`pickup_datetime_id`, `pickup_location_id`, `dropoff_datetime_id`, `airport_fee`, `tip_amount`].;
'Project [vendor_id#459, get_pickup_datetime_id('pickup_datetime)#1293 AS pickup_datetime_id#1294, dropoff_datetime_id#942, pickup_location_id#559, dropoff_location_id#579, ratecode_id#479, passenger_count#677, trip_distance#119, payment_type_id#499, store_and_fwd_flag#159, fare_amount#239, extra#259, mta_tax#279, tip_amount#299, tolls_amount#319, improvement_surcharge#339, congestion_surcharge#599, airport_fee#619, total_amount#359]
+- Project [vendor_id#459, pickup_datetime_id#739, dropoff_datetime_id#942, pickup_location_id#559, dropoff_location_id#579, ratecode_id#479, passenger_count#677, trip_distance#119, payment_type_id#499, store_and_fwd_flag#159, fare_amount#239, extra#259, mta_tax#279, tip_amount#299, tolls_amou

In [ ]:
dropoff_datetime_dim = df.select(['dropoff_datetime']) \
    .distinct() \
    .withColumn('dropoff_datetime_id', index_id(col('dropoff_datetime'))) \
    .withColumn('dropoff_hour', F.hour(col('dropoff_datetime'))) \
    .withColumn('dropoff_day', F.dayofmonth(col('dropoff_datetime'))) \
    .withColumn('dropoff_month', F.month(col('dropoff_datetime'))) \
    .withColumn('dropoff_year', F.year(col('dropoff_datetime'))) \
    .withColumn('dropoff_weekday', F.date_format(col('dropoff_datetime'), 'EEEE'))

dropoff_datetime_dim = dropoff_datetime_dim.select(
    'dropoff_datetime_id',
    'dropoff_datetime',
    'dropoff_hour',
    'dropoff_day',
    'dropoff_month',
    'dropoff_year',
    'dropoff_weekday'
)

In [19]:
dropoff_datetime_id_dict = {row['dropoff_datetime']: row['dropoff_datetime_id'] for row in dropoff_datetime_dim.collect()} 

dropoff_datetime_id_broadcast = sc.broadcast(dropoff_datetime_id_dict)

def get_dropoff_datetime_id(date_time):
    return dropoff_datetime_id_broadcast.value.get(date_time, None)

get_dropoff_datetime_id_udf = F.udf(get_dropoff_datetime_id, types.IntegerType())

df = df.withColumn('dropoff_datetime_id', get_dropoff_datetime_id_udf(col('dropoff_datetime')))

dropoff_datetime_id_broadcast.unpersist()

In [20]:
df = df.select(['vendor_id',
 'pickup_datetime_id',
 'dropoff_datetime_id',
 'pickup_location_id',
 'dropoff_location_id',
 'ratecode_id',
 'passenger_count',
 'trip_distance',
 'payment_type_id',
 'store_and_fwd_flag',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'congestion_surcharge',
 'airport_fee',
 'total_amount'])

In [36]:


dropoff_datetime_dim = dropoff_datetime_dim.withColumn('new_id', new_func(F.col('dropoff_datetime')))

+-------------------+-------------------+------------+-----------+-------------+------------+---------------+--------------+
|dropoff_datetime_id|   dropoff_datetime|dropoff_hour|dropoff_day|dropoff_month|dropoff_year|dropoff_weekday|        new_id|
+-------------------+-------------------+------------+-----------+-------------+------------+---------------+--------------+
|                  0|2019-01-01 01:02:07|           1|          1|            1|        2019|        Tuesday|20190101010207|
|                  1|2019-01-01 01:10:23|           1|          1|            1|        2019|        Tuesday|20190101011023|
|                  2|2019-01-01 01:18:53|           1|          1|            1|        2019|        Tuesday|20190101011853|
|                  3|2019-01-01 00:44:09|           0|          1|            1|        2019|        Tuesday|20190101004409|
|                  4|2019-01-01 00:47:14|           0|          1|            1|        2019|        Tuesday|20190101004714|
